In [ ]:
git clone https://github.com/HRNet/HRNet-Semantic-Segmentation.git /content/HRNet
!wget -O hrnet.pth "https://github.com/hsfzxjy/models.storage/releases/download/HRNet-OCR/hrnet_cs_8090_torch11.pth"

In [2]:
import os
from dataclasses import dataclass
from typing import List, Tuple, Dict, Any, Optional
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
!pip install yacs

In [ ]:
PRETRAINED_HRNET_PATH = "HRNet_W18_C_ssld_pretrained.pth"
CONFIG_YAML_PATH = "seg_hrnet_w18_binary_512x512.yaml"

In [3]:
import os
from PIL import Image

IMAGE_EXTS = {'.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff', '.webp'}

def is_image(path):
    if os.path.splitext(path)[1].lower() not in IMAGE_EXTS:
        return False
    try:
        with Image.open(path) as img:
            img.verify()
        return True
    except Exception:
        return False

import cv2
import numpy as np
import os
from glob import glob

images_folder_path = "New_Labels"
masks_folder_path  = "New_masks"

X = []
Y = []

def first_image_in_folder(folder):
    """Return first image path in folder, or None."""
    for entry in os.scandir(folder):
        if entry.is_file() and is_image(entry.path):
            return entry.path
    return None

count = 0
for subfolder in os.listdir(images_folder_path):

    img_subdir = os.path.join(images_folder_path, subfolder)
    mask_subdir = os.path.join(masks_folder_path, subfolder)

    if not os.path.isdir(img_subdir):
        continue

    count+=1

    print("Processing image:", count)

    img_paths = [entry.path for entry in os.scandir(img_subdir)
                 if entry.is_file() and is_image(entry.path)]

    for p in img_paths:
        X.append(cv2.imread(p))

    if len(img_paths) == 0:
        continue

    ref_img = cv2.imread(img_paths[0])
    H, W = ref_img.shape[:2]

    labels_tensor = np.zeros((H, W), dtype=bool)

    mask_paths = glob(os.path.join(mask_subdir, "**/*.npy"), recursive=True)

    for mp in mask_paths:
        m = np.load(mp)
        labels_tensor |= (m.astype(bool))

    Y.append(labels_tensor)


Processing: 1
Processing: 2
Processing: 3
Processing: 4
Processing: 5
Processing: 6
Processing: 7
Processing: 8
Processing: 9
Processing: 10
Processing: 11
Processing: 12
Processing: 13
Processing: 14
Processing: 15
Processing: 16
Processing: 17
Processing: 18
Processing: 19
Processing: 20
Processing: 21
Processing: 22
Processing: 23
Processing: 24
Processing: 25
Processing: 26
Processing: 27
Processing: 28
Processing: 29
Processing: 30
Processing: 31
Processing: 32
Processing: 33
Processing: 34
Processing: 35
Processing: 36
Processing: 37
Processing: 38
Processing: 39
Processing: 40
Processing: 41
Processing: 42
Processing: 43
Processing: 44
Processing: 45
Processing: 46
Processing: 47
Processing: 48
Processing: 49
Processing: 50
Processing: 51
Processing: 52
Processing: 53
Processing: 54
Processing: 55
Processing: 56
Processing: 57
Processing: 58
Processing: 59
Processing: 60
Processing: 61
Processing: 62
Processing: 63
Processing: 64
Processing: 65
Processing: 66
Processing: 67
Proc

In [4]:
import os
from dataclasses import dataclass
from typing import Optional, List, Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

class BinarySegDataset(Dataset):
    """
    items: list of (image_np, mask_np)
      image_np: HxWxC numpy
      mask_np : HxW numpy with {0,1}
    """
    def __init__(self, items, transform=None):
        self.items = items
        self.transform = transform

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        image_np, mask_np = self.items[idx]

        if self.transform:
            out = self.transform(image=image_np, mask=mask_np)
            image_np, mask_np = out["image"], out["mask"]

        image = torch.from_numpy(image_np).permute(2, 0, 1).float()
        mask  = torch.from_numpy(mask_np).long()

        return image, mask

def pad_collate_fn(batch, pad_value_img=0.0, pad_value_mask=0):
    """
    batch: list of (image, mask)
      image: (C,H,W)
      mask : (H,W)
    returns:
      images: (B,C,Hmax,Wmax)
      masks : (B,Hmax,Wmax)
      sizes : list of original (H,W)
    """
    images, masks = zip(*batch)
    sizes = [(img.shape[-2], img.shape[-1]) for img in images]

    Hmax = max(h for h, w in sizes)
    Wmax = max(w for h, w in sizes)

    padded_images = []
    padded_masks = []

    for img, msk in zip(images, masks):
        h, w = img.shape[-2], img.shape[-1]
        pad_h = Hmax - h
        pad_w = Wmax - w

        img_pad = F.pad(img, (0, pad_w, 0, pad_h), value=pad_value_img)
        msk_pad = F.pad(msk, (0, pad_w, 0, pad_h), value=pad_value_mask)

        padded_images.append(img_pad)
        padded_masks.append(msk_pad)

    images = torch.stack(padded_images, dim=0)
    masks  = torch.stack(padded_masks, dim=0)
    return images, masks, sizes

    def __init__(self, eps=1e-6):
        super().__init__()
        self.eps = eps

    def forward(self, logits, targets):
        """
        logits: (B,1,H,W)
        targets: (B,H,W) {0,1}
        """
        probs = torch.sigmoid(logits)
        targets = targets.unsqueeze(1).float()

        num = 2 * (probs * targets).sum(dim=(2, 3))
        den = (probs + targets).sum(dim=(2, 3)) + self.eps
        dice = 1 - (num / den)
        return dice.mean()

def load_checkpoint(
    model,
    ckpt_path: str,
    strict: bool = False,
    map_location="cpu",
    drop_head: bool = True,
):
    ckpt = torch.load(ckpt_path, map_location=map_location)
    state_dict = ckpt.get("state_dict", ckpt.get("model", ckpt))

    new_state = {k.replace("module.", ""): v for k, v in state_dict.items()}

    if drop_head:
        head_keys = ["last_layer.3.weight", "last_layer.3.bias"]

        head_keys += [k for k in new_state.keys() if "aux" in k or "classifier" in k]

        removed = []
        for k in head_keys:
            if k in new_state:
                removed.append(k)
                new_state.pop(k)

        if removed:
            print("Dropped head keys from checkpoint:", removed)

        model_sd = model.state_dict()
        mismatched = []
        for k in list(new_state.keys()):
            if k in model_sd and new_state[k].shape != model_sd[k].shape:
                mismatched.append((k, tuple(new_state[k].shape), tuple(model_sd[k].shape)))
                new_state.pop(k)

        if mismatched:
            print("Dropped mismatched keys:")
            for k, shp_ckpt, shp_model in mismatched:
                print(f"  {k}: ckpt {shp_ckpt} vs model {shp_model}")

    missing, unexpected = model.load_state_dict(new_state, strict=strict)
    print(f"Loaded checkpoint: {ckpt_path}")
    if missing:
        print("Missing keys:", missing)
    if unexpected:
        print("Unexpected keys:", unexpected)

    return ckpt

@dataclass
class TrainConfig:
    device: str = "cuda"
    lr: float = 1e-4
    weight_decay: float = 1e-4
    epochs: int = 20
    grad_clip: float = 1.0
    amp: bool = True
    bce_weight: float = 1.0
    dice_weight: float = 1.0
    log_every: int = 20


def train_hrnet_binary_seg(
    model: nn.Module,
    train_loader: DataLoader,
    val_loader: Optional[DataLoader],
    ckpt_path: str,
    save_dir: str,
    cfg: TrainConfig
):
    os.makedirs(save_dir, exist_ok=True)
    device = torch.device(cfg.device)
    model.to(device)

    load_checkpoint(model, ckpt_path, strict=False, map_location=device)


    for p in model.parameters():
        p.requires_grad = False

    for p in model.last_layer.parameters():
      p.requires_grad = True

    optimizer = torch.optim.AdamW(model.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=cfg.epochs)

    bce_loss = nn.BCEWithLogitsLoss()
    dice_loss = DiceLoss()

    scaler = torch.cuda.amp.GradScaler(enabled=cfg.amp and device.type == "cuda")

    best_val = float("inf")

    for epoch in range(cfg.epochs):
        model.train()
        running = 0.0

        if epoch > 5:
          for name, p in model.named_parameters():
            if "stage4" in name or "last_layer" in name:
                p.requires_grad = True
            else:
                p.requires_grad = False

        for it, batch in enumerate(train_loader):
            images, masks, _sizes = batch

            images = images.to(device, non_blocking=True)
            masks  = masks.to(device, non_blocking=True)

            optimizer.zero_grad(set_to_none=True)

            with torch.cuda.amp.autocast(enabled=cfg.amp and device.type == "cuda"):
                logits = model(images)
                if logits.dim() == 3:
                    logits = logits.unsqueeze(1)

                if logits.shape[-2:] != masks.shape[-2:]:
                    logits = F.interpolate(
                        logits, size=masks.shape[-2:], mode="bilinear", align_corners=False
                    )

                loss_bce  = bce_loss(logits, masks.unsqueeze(1).float())
                loss_dice = dice_loss(logits, masks)
                loss = cfg.bce_weight * loss_bce + cfg.dice_weight * loss_dice

            scaler.scale(loss).backward()

            if cfg.grad_clip is not None:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), cfg.grad_clip)

            scaler.step(optimizer)
            scaler.update()

            running += loss.item()
            if (it + 1) % cfg.log_every == 0:
                avg = running / cfg.log_every
                print(f"Epoch {epoch+1}/{cfg.epochs} | iter {it+1}/{len(train_loader)} | loss {avg:.4f}")
                running = 0.0

        scheduler.step()

        val_loss = None
        if val_loader is not None:
            val_loss = evaluate(model, val_loader, bce_loss, dice_loss, device)
            print(f"Epoch {epoch+1} validation loss: {val_loss:.4f}")

            if val_loss < best_val:
                best_val = val_loss
                torch.save(
                    {"model": model.state_dict(), "epoch": epoch+1, "val_loss": val_loss},
                    os.path.join(save_dir, "best.pt")
                )
                print("Saved best checkpoint.")

        torch.save(
            {"model": model.state_dict(), "epoch": epoch+1, "val_loss": val_loss},
            os.path.join(save_dir, "last.pt")
        )


@torch.no_grad()
def evaluate(model, loader, bce_loss, dice_loss, device):
    model.eval()
    total = 0.0
    n = 0

    for images, masks, _sizes in loader:
        images = images.to(device, non_blocking=True)
        masks  = masks.to(device, non_blocking=True)

        logits = model(images)
        if logits.dim() == 3:
            logits = logits.unsqueeze(1)

        if logits.shape[-2:] != masks.shape[-2:]:
            logits = F.interpolate(
                logits, size=masks.shape[-2:], mode="bilinear", align_corners=False
            )

        loss = bce_loss(logits, masks.unsqueeze(1).float()) + dice_loss(logits, masks)

        total += loss.item()
        n += 1

    return total / max(n, 1)

class InferenceDataset(Dataset):
    """
    images_np: list of numpy images HxWxC
    """
    def __init__(self, images_np, transform=None):
        self.images = images_np
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_np = self.images[idx]
        if self.transform:
            out = self.transform(image=img_np)
            img_np = out["image"]

        img_t = torch.from_numpy(img_np).permute(2,0,1).float()
        return img_t


def pad_images_collate_fn(batch, pad_value=0.0):
    sizes = [(img.shape[-2], img.shape[-1]) for img in batch]
    Hmax = max(h for h, w in sizes)
    Wmax = max(w for h, w in sizes)

    padded = []
    for img in batch:
        h, w = img.shape[-2], img.shape[-1]
        pad_h, pad_w = Hmax - h, Wmax - w
        padded.append(F.pad(img, (0, pad_w, 0, pad_h), value=pad_value))

    return torch.stack(padded, dim=0), sizes


def load_finetuned(model, ckpt_path, device):
    ckpt = torch.load(ckpt_path, map_location=device)
    state_dict = ckpt.get("model", ckpt.get("state_dict", ckpt))
    new_state = {k.replace("module.", ""): v for k, v in state_dict.items()}
    model.load_state_dict(new_state, strict=False)
    model.to(device).eval()
    return model


@torch.no_grad()
def infer_batch(
    model: nn.Module,
    images_np: List[np.ndarray],
    ckpt_path: str,
    device: str = "cuda",
    batch_size: int = 4,
    threshold: float = 0.5,
    transform=None
):
    device = torch.device(device)
    model = load_finetuned(model, ckpt_path, device)

    ds = InferenceDataset(images_np, transform=transform)
    loader = DataLoader(
        ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=0,
        pin_memory=(device.type=="cuda"),
        collate_fn=pad_images_collate_fn
    )

    all_masks = []
    all_probs = []

    for imgs_pad, sizes in loader:
        imgs_pad = imgs_pad.to(device)

        logits = model(imgs_pad)
        if logits.dim() == 3:
            logits = logits.unsqueeze(1)

        probs = torch.sigmoid(logits)

        for i, (H, W) in enumerate(sizes):
            p = probs[i, 0, :H, :W].cpu().numpy()
            all_probs.append(p)
            all_masks.append((p >= threshold).astype(np.uint8))

    return all_masks, all_probs


In [5]:
def load_hrnet_backbone_only(model, ckpt_path):
    ckpt = torch.load(ckpt_path, map_location="cpu")
    sd = ckpt.get("state_dict", ckpt.get("model", ckpt))
    sd = {k.replace("module.", ""): v for k, v in sd.items()}
    for k in ["last_layer.3.weight", "last_layer.3.bias"]:
        sd.pop(k, None)
    return model.load_state_dict(sd, strict=False)


In [10]:
import sys
sys.path.append("HRNet/lib")
sys.path.append("HRNet")

import numpy as np
if not hasattr(np, "int"):   np.int   = int
if not hasattr(np, "bool"):  np.bool  = bool
if not hasattr(np, "float"): np.float = float

from argparse import Namespace
from config import config, update_config
from models.seg_hrnet import HighResolutionNet

import torch
import torch.nn as nn

cfg_file = CONFIG_YAML_PATH
args = Namespace(cfg=cfg_file, opts=[])
update_config(config, args)

model = HighResolutionNet(config)

old_conv = model.last_layer[3]  # Conv2d(720, 19)
model.last_layer[3] = nn.Conv2d(
    old_conv.in_channels, 1, kernel_size=1, stride=1, padding=0
)

pretrained = PRETRAINED_HRNET_PATH
ckpt = torch.load(pretrained, map_location="cpu")

state_dict = ckpt.get("state_dict", ckpt.get("model", ckpt))
state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}

head_keys = ["last_layer.3.weight", "last_layer.3.bias"]
for k in head_keys:
    state_dict.pop(k, None)

missing, unexpected = model.load_state_dict(state_dict, strict=False)

print("Loaded pretrained backbone.")
print("Missing keys (expected):", missing)
print("Unexpected keys:", unexpected)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print("Model ready on", device)


Loaded pretrained backbone.
Missing keys (expected): ['last_layer.0.weight', 'last_layer.0.bias', 'last_layer.1.weight', 'last_layer.1.bias', 'last_layer.1.running_mean', 'last_layer.1.running_var', 'last_layer.3.weight', 'last_layer.3.bias']
Unexpected keys: ['incre_modules.0.0.conv1.weight', 'incre_modules.0.0.bn1.weight', 'incre_modules.0.0.bn1.bias', 'incre_modules.0.0.bn1.running_mean', 'incre_modules.0.0.bn1.running_var', 'incre_modules.0.0.conv2.weight', 'incre_modules.0.0.bn2.weight', 'incre_modules.0.0.bn2.bias', 'incre_modules.0.0.bn2.running_mean', 'incre_modules.0.0.bn2.running_var', 'incre_modules.0.0.conv3.weight', 'incre_modules.0.0.bn3.weight', 'incre_modules.0.0.bn3.bias', 'incre_modules.0.0.bn3.running_mean', 'incre_modules.0.0.bn3.running_var', 'incre_modules.0.0.downsample.0.weight', 'incre_modules.0.0.downsample.1.weight', 'incre_modules.0.0.downsample.1.bias', 'incre_modules.0.0.downsample.1.running_mean', 'incre_modules.0.0.downsample.1.running_var', 'incre_modul

In [37]:
from sklearn.model_selection import train_test_split

import torch
torch.cuda.empty_cache()

assert len(X) == len(Y)

X_train, X_val, Y_train, Y_val = train_test_split(
    X, Y, test_size=0.2, random_state=42, shuffle=True
)

train_items = list(zip(X_train, Y_train))
val_items   = list(zip(X_val,   Y_val))

train_ds = BinarySegDataset(train_items, transform=None)  # or train_tfms
val_ds   = BinarySegDataset(val_items,   transform=None)  # or val_tfms

train_loader = DataLoader(
    train_ds,
    batch_size=3,
    shuffle=True,
    num_workers=3,
    pin_memory=True,
    collate_fn=pad_collate_fn
)

val_loader = DataLoader(
    val_ds,
    batch_size=3,
    shuffle=False,
    num_workers=3,
    pin_memory=True,
    collate_fn=pad_collate_fn
)

cfg = TrainConfig(device="cuda", epochs=30, lr=1e-4)

train_hrnet_binary_seg(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    ckpt_path="runs_h18/hrnet_18.pt",
    save_dir="runs_h18",
    cfg=cfg
)


Dropped head keys from checkpoint: ['last_layer.3.weight', 'last_layer.3.bias']
Loaded checkpoint: /content/runs_h18/best.pt
Missing keys: ['last_layer.3.weight', 'last_layer.3.bias']


/tmp/ipython-input-2753826384.py:204: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=cfg.amp and device.type == "cuda")
/tmp/ipython-input-2753826384.py:227: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=cfg.amp and device.type == "cuda"):


Epoch 1/30 | iter 20/134 | loss 0.0854
Epoch 1/30 | iter 40/134 | loss 0.0674
Epoch 1/30 | iter 60/134 | loss 0.0710
Epoch 1/30 | iter 80/134 | loss 0.0739
Epoch 1/30 | iter 100/134 | loss 0.0705
Epoch 1/30 | iter 120/134 | loss 0.0658
Epoch 1 validation loss: 0.1496
Saved best checkpoint.
Epoch 2/30 | iter 20/134 | loss 0.0707
Epoch 2/30 | iter 40/134 | loss 0.0781
Epoch 2/30 | iter 60/134 | loss 0.0705
Epoch 2/30 | iter 80/134 | loss 0.0767
Epoch 2/30 | iter 100/134 | loss 0.0738
Epoch 2/30 | iter 120/134 | loss 0.0688
Epoch 2 validation loss: 0.1521
Epoch 3/30 | iter 20/134 | loss 0.0751
Epoch 3/30 | iter 40/134 | loss 0.0709
Epoch 3/30 | iter 60/134 | loss 0.0793
Epoch 3/30 | iter 80/134 | loss 0.0734
Epoch 3/30 | iter 100/134 | loss 0.0831
Epoch 3/30 | iter 120/134 | loss 0.0729
Epoch 3 validation loss: 0.1478
Saved best checkpoint.
Epoch 4/30 | iter 20/134 | loss 0.0787
Epoch 4/30 | iter 40/134 | loss 0.0671
Epoch 4/30 | iter 60/134 | loss 0.0721
Epoch 4/30 | iter 80/134 | loss 0

In [33]:
import os
import torch
import torch.nn.functional as F
import numpy as np
from PIL import Image

def save_all_val_predictions(
    model,
    val_loader,
    best_ckpt_path,
    save_dir,
    threshold=0.5,
    device="cuda"
):
    device = torch.device(device if torch.cuda.is_available() else "cpu")

    # ---- load best checkpoint once ----
    ckpt = torch.load(best_ckpt_path, map_location=device)
    state_dict = ckpt.get("model", ckpt.get("state_dict", ckpt))
    state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}
    model.load_state_dict(state_dict, strict=False)
    model.to(device).eval()

    os.makedirs(save_dir, exist_ok=True)

    global_idx = 0

    with torch.no_grad():
        for batch_idx, batch in enumerate(val_loader):
            images, masks, sizes = batch   # images: (B, C, Hpad, Wpad)
            images = images.to(device)

            # ---- forward ----
            logits = model(images)
            if logits.dim() == 3:
                logits = logits.unsqueeze(1)

            # upsample if needed
            if logits.shape[-2:] != masks.shape[-2:]:
                logits = F.interpolate(
                    logits, size=masks.shape[-2:], mode="bilinear", align_corners=False
                )

            probs = torch.sigmoid(logits)

            # ---- per-sample loop ----
            for i in range(images.size(0)):
                H, W = sizes[i]  # original image size

                # ----- save original input image -----
                # ----- save original input image (robust float handling) -----
                img_t = images[i, :, :H, :W].detach().cpu()   # (C,H,W), float

                # If normalized or arbitrary float range, bring to 0..1 safely
                img_t = img_t.float()
                img_t = img_t - img_t.min()
                if img_t.max() > 0:
                    img_t = img_t / img_t.max()

                img_np = (img_t.numpy() * 255).astype(np.uint8)  # uint8 CHW

                # CHW -> HWC for RGB, or squeeze for grayscale
                if img_np.shape[0] == 1:
                    img_np = img_np[0]  # (H,W)
                else:
                    img_np = np.transpose(img_np, (1, 2, 0))  # (H,W,C)

                img_save_path = os.path.join(save_dir, f"img{global_idx}.png")
                Image.fromarray(img_np).save(img_save_path)

                global_idx += 1

    print(f"Saved {global_idx} images + masks to: {save_dir}")



best_ckpt = "runs_h18/best.pt"
save_dir = "runs_h18/val_preds"

save_all_val_predictions(
    model=model,
    val_loader=val_loader,
    best_ckpt_path=best_ckpt,
    save_dir=save_dir,
    threshold=0.5,
    device=cfg.device
)



Saved 100 images + masks to: /content/runs_h18/val_preds


In [34]:
!zip -r validation_masks.zip /content/runs_h18/val_preds

updating: content/runs_h18/val_preds/ (stored 0%)
updating: content/runs_h18/val_preds/val_pred_00077.png (deflated 6%)
updating: content/runs_h18/val_preds/val_pred_00028.png (deflated 5%)
updating: content/runs_h18/val_preds/val_pred_00081.png (deflated 1%)
updating: content/runs_h18/val_preds/val_pred_00062.png (deflated 6%)
updating: content/runs_h18/val_preds/val_pred_00023.png (stored 0%)
updating: content/runs_h18/val_preds/val_pred_00045.png (deflated 2%)
updating: content/runs_h18/val_preds/val_pred_00090.png (stored 0%)
updating: content/runs_h18/val_preds/val_pred_00072.png (stored 0%)
updating: content/runs_h18/val_preds/val_pred_00097.png (stored 0%)
updating: content/runs_h18/val_preds/val_pred_00036.png (deflated 5%)
updating: content/runs_h18/val_preds/val_pred_00037.png (stored 0%)
updating: content/runs_h18/val_preds/val_pred_00095.png (deflated 5%)
updating: content/runs_h18/val_preds/val_pred_00078.png (stored 0%)
updating: content/runs_h18/val_preds/val_pred_00049.